# claudette-agent Examples

This notebook demonstrates all the features of `claudette-agent`, a Claude Agent SDK wrapper with a Claudette-compatible API.

## Setup

Make sure you have `claude-agent-sdk` installed and configured with your Claude Code subscription.

In [ ]:
# Install if needed (uncomment)
# !pip install claude-agent-sdk pydantic

# Import the package
from claudette_agent import (
    Chat, AsyncChat, Client,
    contents, query, tool,
    MCPToolkit, mcp_tool, create_mcp_server,
    Usage, Message, TextBlock
)

# Default model to use
MODEL = "claude-sonnet-4-5-20250929"

print("claudette-agent loaded successfully!")

## 1. Basic Chat

The `Chat` class maintains conversation history and supports system prompts.

In [ ]:
# Create a chat with a system prompt
chat = Chat(model=MODEL, sp="You are a helpful assistant. Be concise.")

# Send a message
response = await chat("What is the capital of France?")
print("Response:", contents(response))

In [ ]:
# Continue the conversation - Claude remembers the context
response = await chat("What is its population?")
print("Follow-up:", contents(response))

In [ ]:
# View conversation history
print(f"History length: {len(chat.h)} messages")
for i, msg in enumerate(chat.h):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])
    if isinstance(content, list) and content:
        text = content[0].get('text', '')[:50]
    else:
        text = str(content)[:50]
    print(f"  {i+1}. [{role}]: {text}...")

## 2. Simple Query (No History)

Use `query()` for one-shot queries without conversation state.

In [ ]:
# Simple one-shot query
response = await query("What is 2 + 2?", model=MODEL)
print("Answer:", contents(response))

## 3. Using Tools

Define tools using the `@tool` decorator and pass them to `Chat`.

In [ ]:
@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression safely."""
    # Simple calculator - only allow safe operations
    allowed = set('0123456789+-*/(). ')
    if all(c in allowed for c in expression):
        return str(eval(expression))
    return "Error: Invalid expression"

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    # Simulated weather data
    weather_data = {
        "paris": "Sunny, 22°C",
        "london": "Cloudy, 15°C",
        "tokyo": "Rainy, 18°C",
        "new york": "Clear, 25°C",
    }
    return weather_data.get(city.lower(), f"Weather data not available for {city}")

print("Tools defined!")

In [ ]:
# Create chat with tools
chat_with_tools = Chat(
    model=MODEL,
    sp="You are a helpful assistant with access to a calculator and weather service.",
    tools=[calculate, get_weather]
)

# Ask a math question
response = await chat_with_tools("What is 15 * 23 + 7?")
print("Math answer:", contents(response))

In [ ]:
# Ask about weather
response = await chat_with_tools("What's the weather in Paris?")
print("Weather:", contents(response))

## 4. Tool Loop (Automatic Tool Following)

Use `toolloop()` to automatically follow up on tool calls.

In [ ]:
@tool
def search_database(query: str) -> str:
    """Search a database for information."""
    # Simulated search results
    results = {
        "python": "Python is a high-level programming language known for its simplicity.",
        "async": "Async programming allows concurrent execution without threads.",
        "ai": "AI is the simulation of human intelligence by machines.",
    }
    for key, value in results.items():
        if key in query.lower():
            return value
    return "No results found."

# Create chat with search tool
research_chat = Chat(
    model=MODEL,
    sp="You are a research assistant. Use the search tool to find information.",
    tools=[search_database]
)

# Run toolloop - automatically follows tool calls
results = await research_chat.toolloop(
    "Search for information about Python and summarize it.",
    max_steps=3
)

print("Toolloop results:")
for i, result in enumerate(results):
    print(f"\n--- Step {i+1} ---")
    print(contents(result)[:200] + "..." if len(contents(result)) > 200 else contents(result))

## 5. Structured Outputs with Pydantic

Use `chat.struct()` to get responses as Pydantic models.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="The person's full name")
    age: int = Field(description="The person's age in years")
    occupation: str = Field(description="The person's job or profession")
    hobbies: Optional[List[str]] = Field(default=None, description="List of hobbies")

print("Pydantic model defined!")

In [ ]:
# Create a chat for structured extraction
struct_chat = Chat(model=MODEL)

# Extract structured data
person = await struct_chat.struct(
    "Extract the person info: John Smith is a 35-year-old software engineer who enjoys hiking and photography.",
    Person
)

print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"Occupation: {person.occupation}")
print(f"Hobbies: {person.hobbies}")

In [ ]:
# More complex structured output
class Recipe(BaseModel):
    """A cooking recipe."""
    name: str
    ingredients: List[str]
    steps: List[str]
    prep_time_minutes: int
    difficulty: str

recipe_chat = Chat(model=MODEL)
recipe = await recipe_chat.struct(
    "Create a simple pasta recipe with tomato sauce.",
    Recipe
)

print(f"Recipe: {recipe.name}")
print(f"Prep time: {recipe.prep_time_minutes} minutes")
print(f"Difficulty: {recipe.difficulty}")
print(f"\nIngredients:")
for ing in recipe.ingredients:
    print(f"  - {ing}")
print(f"\nSteps:")
for i, step in enumerate(recipe.steps, 1):
    print(f"  {i}. {step}")

## 6. Streaming Responses

Use `chat.stream()` to get responses as they're generated.

**Important:** The Claude Agent SDK streams complete message blocks, not individual text characters like the Anthropic API. Each yielded value is a complete text block from Claude's response.

In [ ]:
# Create a chat for streaming
stream_chat = Chat(model=MODEL, sp="You are a storyteller. Tell engaging short stories.")

print("Streaming response (note: SDK yields message blocks, not characters):\n")
async for block in stream_chat.stream("Tell me a very short story about a robot learning to paint."):
    print(block, end="", flush=True)

print("\n\n--- Done! ---")

## 7. Cost Tracking

Track token usage and estimated costs.

In [ ]:
# Create a new chat to track usage
tracked_chat = Chat(model=MODEL)

# Make several requests
await tracked_chat("What is Python?")
await tracked_chat("What are its main uses?")
await tracked_chat("Give me a simple code example.")

# Check usage statistics
print("Usage Statistics:")
print(f"  Input tokens: {tracked_chat.use.input_tokens:,}")
print(f"  Output tokens: {tracked_chat.use.output_tokens:,}")
print(f"  Total tokens: {tracked_chat.use.total:,}")
print(f"  Estimated cost: ${tracked_chat.cost:.6f}")

## 8. Low-Level Client

Use `Client` directly for more control.

In [ ]:
# Create a client
client = Client(model=MODEL)

# Make a direct call
response = await client(
    msgs="Explain quantum computing in one sentence.",
    sp="You are a physics professor. Be concise.",
    maxtok=100
)

print("Response:", contents(response))
print(f"\nTokens used: {client.use.total}")

## 9. MCP Server Integration (Advanced)

Create and use MCP servers for complex tool integrations.

In [ ]:
# Define MCP tools
@mcp_tool("add", "Add two numbers", {"a": float, "b": float})
async def add_numbers(args):
    result = args['a'] + args['b']
    return {"content": [{"type": "text", "text": str(result)}]}

@mcp_tool("multiply", "Multiply two numbers", {"a": float, "b": float})
async def multiply_numbers(args):
    result = args['a'] * args['b']
    return {"content": [{"type": "text", "text": str(result)}]}

# Note: Full MCP server usage requires the claude-agent-sdk to be properly configured
# This example shows the API but may not run without full SDK setup
print("MCP tools defined!")
print("Note: Full MCP functionality requires claude-agent-sdk configuration.")

## Summary

This notebook covered:

1. **Basic Chat** - `Chat` class with conversation history
2. **Simple Query** - `query()` for one-shot requests
3. **Tools** - `@tool` decorator for function calling
4. **Tool Loop** - `toolloop()` for automatic tool following
5. **Structured Outputs** - `struct()` with Pydantic models
6. **Streaming** - `stream()` for real-time responses
7. **Cost Tracking** - `use` and `cost` properties
8. **Client** - Low-level `Client` class
9. **MCP** - Model Context Protocol integration

For more information, see the [README](./README.md) and the [claude-agent-sdk documentation](https://github.com/anthropics/claude-agent-sdk-python).